In [1]:
import pandas as pd
import geopandas as gpd
import networkx as nx

from netana.graph_manager.network import create_gdf_network, split_intersecting_lines
from netana.graph_manager.graph_creation import graph_from_gdf

c:\Users\Compumar\Documents\DESA\GIS_NetworkAn\envNetworkAnalysis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# LOAD DATA

In [2]:
# Partido de La Matanza
gdf_mask_LA_MATANZA = gpd.read_file("./output/LIMITE_POLITICO.shp", where="DISTRITO='LA MATANZA'")
gdf_mask_LA_MATANZA

,SUPERFICIE,DISTRITO,geometry
0,327,LA MATANZA,"POLYGON ((-58.55620 -34.63474, -58.55573 -34.6..."


### Create streets graph

In [3]:
# street layer
gdf_streets = gpd.read_file("./output/Calles_topo_timetravel.shp", mask=gdf_mask_LA_MATANZA)
gdf_streets = gpd.clip(gdf_streets, gdf_mask_LA_MATANZA)

C:\Users\Compumar\AppData\Local\Temp\ipykernel_3180\3132287212.py:3: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84" ...

  gdf_streets = gpd.clip(gdf_streets, gdf_mask_LA_MATANZA)


In [4]:
gdf_streets_network_lines, gdf_streets_network_points = split_intersecting_lines(gdf_streets)
gdf_streets = create_gdf_network(gdf_streets, "3857", 80, v_label='street')
gdf_streets.head()

,id,len_m,weight,label,geometry
0,34600,46.04,0.575500,street,"LINESTRING (-6531759.272 -4150996.664, -653173..."
1,25549,174.04,2.175500,street,"LINESTRING (-6531802.205 -4150806.933, -653167..."
2,25944,533.05,6.663125,street,"LINESTRING (-6531802.205 -4150806.933, -653150..."
3,26066,171.86,2.148250,street,"LINESTRING (-6531329.010 -4150561.688, -653120..."
4,25756,320.38,4.004750,street,"LINESTRING (-6531195.975 -4150496.412, -653096..."


In [5]:
G, gdf_nodes, gdf_edges, dict_mapping_nodes = graph_from_gdf(gdf_streets, f_weight='weight')

# LOCATIONS

In [6]:
from netana.service_analysis.sources import create_sources_from_gdf, closest_nodes_to_sources

In [7]:
gdf_sources = gpd.read_file("./output/Establecimientos_salud_publicos_alta_al20220721.shp", where="cat='Hospital'", mask=gdf_mask_LA_MATANZA)
gdf_sources = create_sources_from_gdf(gdf_sources, "3857")
print(len(gdf_sources))
list_node_sources, gdf_locations = closest_nodes_to_sources(gdf_sources, gdf_nodes, field_category="cat", field_label="gna", field_name="fna")
list_node_sources

9


['15829_street',
 '51417_street',
 '41998_street',
 '57810_street',
 '48981_street',
 '12726_street',
 '52470_street',
 '44375_street',
 '27873_street']

In [42]:
gdf_locations.head()

,id,lat,long,fna,gna,nam,sag,cpd,cnr,cde,...,tel,sse,nrs,cp1,nor,mod,mai,geometry,index_right,node_id
0,loc_0,-34.769939,-58.648008,Hospital Materno Infantil Dr. J. Equiza,Hospital Materno Infantil,Dr. J. Equiza,Dis. Ministerio de Salud BA,42700031,1.006427e+13,427,...,02202-422232 int,Público,XII,1759,Hospital Materno Infantil Dr. José Equiza,Con Internación,NaN,POINT (-6528666.364 -4132660.548),15829,15829_street
1,loc_1,-34.682803,-58.585938,Hospital Municipal de Niños de San Justo,Hospital Municipal,de Niños de San Justo,Dis. Ministerio de Salud BA,42700120,1.106427e+13,427,...,011-44419371/2776,Público,XII,1754,Hospital Municipal del Niño San Justo,Con Internación,NaN,POINT (-6521756.792 -4120858.478),51417,51417_street
2,loc_2,-34.753520,-58.580900,Hospital Local Materno Infantil Dra. Germani,Hospital,Local Materno Infantil Dra. Germani,Dis. Ministerio de Salud BA,42700049,1.206427e+13,427,...,011-46260315/0447,Público,XII,1757,Hospital Materno Infantil Dra. Teresa Luisa Ge...,Con Internación,NaN,POINT (-6521196.001 -4130435.784),41998,41998_street
3,loc_3,-34.676989,-58.560969,Policlinico Central,Policlinico,de San Justo,Dis. Ministerio de Salud BA,42700138,1.006427e+13,427,...,011-46510164,Público,XII,1754,Policlínico Central San Justo,Con Internación,NaN,POINT (-6518977.238 -4120071.446),57810,57810_street
4,loc_4,-34.695396,-58.579157,Hosp. Interz. Gral. de Ag. Dr. Paroissien,Hosp. Interz.,Gral. de Ag. Dr. Paroissien,Dis. Ministerio de Salud BA,42700651,1.006427e+13,427,...,011-46693440/4466120,Público,XII,1765,Hospital Interzonal General de Agudos Dr. Paro...,Con Internación,NaN,POINT (-6521001.951 -4122563.325),48981,48981_street


# ISOCHRONES

In [35]:
from netana.isochrones.edges import calculate_isochrone_edges
from netana.isochrones.isochrones import create_isochrones, create_individual_sources_isochrones, create_isochrones_for_multiple_weights

## Isochrones by value

In [9]:
v_time = 15
df_edges_cost_limit = calculate_isochrone_edges(G, gdf_nodes, gdf_edges, 'weight', list_node_sources, v_weight_limit=v_time)
df_edges_cost_limit = df_edges_cost_limit[['node_id_st', 'node_id_en', 'weight', 'w_start', 'w_end', 'w_mean', 'w_limit', 'geometry']]
gdf_buffer = create_isochrones(df_edges_cost_limit, "3857", 5, 100)
gdf_buffer.to_file("./output/isochrones/isochrones_by_value.shp")

In [ ]:
# Second alternative
v_time = 15
gdf_buffer = create_isochrones(G, gdf_nodes, gdf_edges, list_node_sources, v_time, "3857", 5, 100, f_dissolve='w_limit')
gdf_buffer.to_file("./output/isochrones/isochrones_by_value.shp")

## Isochrones by field

In [10]:
print(list_node_sources)
df_node_sources = pd.DataFrame(zip(list_node_sources, [15,15,15,10,10,10,5,5,5]), columns=['node_loc_id', 'weight'])
print(df_node_sources.head())

df_edges_cost_limit = calculate_isochrone_edges(G, gdf_nodes, gdf_edges, 'weight', df_node_sources)
gdf_buffer = create_isochrones(df_edges_cost_limit, "3857", 5, 100)
gdf_buffer.to_file("./output/isochrones/isochrones_by_field.shp")

['15829_street', '51417_street', '41998_street', '57810_street', '48981_street', '12726_street', '52470_street', '44375_street', '27873_street']


,node_loc_id,weight
0,15829_street,15
1,51417_street,15
2,41998_street,15
3,57810_street,10
4,48981_street,10
5,12726_street,10
6,52470_street,5
7,44375_street,5
8,27873_street,5


In [38]:
# Second alternative
gdf_buffer = create_isochrones(G, gdf_nodes, gdf_edges, list_node_sources, v_time, "3857", 5, 100, f_dissolve='w_limit')
gdf_buffer.to_file("./output/isochrones/isochrones_by_field.shp")

NameError: name 'df_node_sources' is not defined

# Individual isochrones

In [10]:
v_time = 15
df_edges_weight_limit = calculate_isochrone_edges(G, gdf_nodes, gdf_edges, "weight", list_node_sources, v_time, iso_type='individual')
df_all_isos = create_isochrones(df_edges_weight_limit[['node_source_st', 'node_source_en', 'geometry']], "3857", 5, 100, f_dissolve='node_source_st')

df_all_isos = df_all_isos[['node_source_en', 'geometry']]
df_all_isos.rename(columns={'node_source_en':'node_id'}, inplace=True)

if 'geometry' in list(gdf_locations.columns):
    gdf_amenities_iso = df_all_isos.merge(gdf_locations.drop(columns='geometry'), how='left', on='node_id')
else:
    gdf_amenities_iso = df_all_isos.merge(gdf_locations, how='left', on='node_id')

gdf_amenities_iso[['node_id', 'geometry']].to_file("./output/isochrones/isochrones_by_value_individual.shp")

In [ ]:
# Second alternative
v_time = 15
gdf_amenities_iso = create_individual_sources_isochrones(G, gdf_nodes, gdf_edges, gdf_locations, list_node_sources, "weight", v_time, v_frac=0, v_time_factor= 1.2)
gdf_amenities_iso[['node_id', 'geometry']].to_file("./output/isochrones/isochrones_by_value_individual.shp")

# Multiple isochrones, ring shape

In [37]:
iso_rings = create_isochrones_for_multiple_weights(
    G, gdf_nodes, gdf_edges, "weight" , [5, 10, 15], list_node_sources, 5, 100, area_type='rings', v_crs_proj="3857")
iso_rings.to_file("./output/isochrones/isochrones_by_value_rings.shp")